# Ejercicio

Trades at close

Comisión: $0.125\%$

$SL=TP=5\%$

Cash: $\$1 M$

N shares: $50$

No taxes

No leverages

Margin acc: $50\%$

Borrow rate: $0.25\%$

Time frame: $5m$

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ta
from dataclasses import dataclass
import optuna
import tqdm as notebook_tqdm

sns.set_theme()

In [11]:
from dataclasses import dataclass

@dataclass
class Operation:
    time: str
    price: float
    stop_loss: float
    take_profit: float
    n_shares: int
    type: str

In [12]:
data = pd.read_csv('aapl_5m_train.csv').dropna()

In [13]:
def get_portfolio_value(cash: float, long_ops: list[Operation], short_ops: list[Operation], 
                        current_price: float, n_shares: int) -> float:
   
    val = cash 

    # Add long position values
    val += len(long_ops) * current_price * n_shares 

    # Add short position values

    return val

In [14]:
def backtest(data, trial) -> float:
    data = data.copy()

    rsi_window = trial.suggest_int("rsi_window", 5, 50)
    rsi_lower = trial.suggest_int("rsi_lower", 5, 35)
    rsi_upper = trial.suggest_int("rsi_upper", 65, 95)
    stop_loss = trial.suggest_float("stop_loss", 0.01, 0.15)
    take_profit = trial.suggest_float("take_profit", 0.01, 0.15)
    n_shares = trial.suggest_int("n_shares", 50, 500)

    rsi_indicator = ta.momentum.RSIIndicator(data.Close, window=rsi_window)

    data["rsi"] = rsi_indicator.rsi()

    historic = data.dropna()
    historic["buy_signal"] = historic.rsi < rsi_lower
    historic["sell_signal"] = historic.rsi > rsi_upper


    COM = 0.125 / 100
    SL = stop_loss
    TP = take_profit
    BORROW_RATE = 0.25/100

    cash = 1_000_000


    active_long_positions= []

    portfolio_value = [cash]

    for i, row in historic.iterrows():
        
        # This only works for long positions
        portfolio_value.append(get_portfolio_value(cash, active_long_positions, [], row.Close, n_shares))


        # Close Operations
        for position in active_long_positions.copy():
            if row.Close > position.take_profit or row.Close < position.stop_loss:
                cash += row.Close * position.n_shares * (1 - COM)
                active_long_positions.remove(position)

        # --- BUY
        # Check Signal
        if not row.buy_signal:
            portfolio_value.append(get_portfolio_value(cash, active_long_positions, [], row.Close, n_shares))

            continue

        # Enough Cash?
        if cash < row.Close * n_shares * (1 + COM):
            portfolio_value.append(get_portfolio_value(cash, active_long_positions, [], row.Close, n_shares))

            continue
        # Discount Costs
        cash -= row.Close * n_shares * (1 + COM)


        # Save the Operation as active position
        active_long_positions.append(
            Operation(
            time=row.Datetime,
            price=row.Close,
            take_profit=row.Close * (1 + TP),
            stop_loss= row.Close * (1 - SL),
            n_shares=n_shares,
            type = "LONG"
        )
    )

        portfolio_value.append(get_portfolio_value(cash, active_long_positions, [], row.Close, n_shares))

    #añadimos al cash las posiciones que siguen abiertas cuando terminamos las posiciones

    cash += row.Close * len(active_long_positions) *(1 - COM)
    active_position = []

    return (cash / 1_000_000) - 1


In [15]:
study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: backtest(data, trial), n_trials=10)

[I 2025-09-12 07:41:22,555] A new study created in memory with name: no-name-15fcc1a5-51a7-43a4-994c-93b246618b6d
/var/folders/39/pg899vtj5774718bjd1zhn6m0000gn/T/ipykernel_31246/2343508439.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historic["buy_signal"] = historic.rsi < rsi_lower
/var/folders/39/pg899vtj5774718bjd1zhn6m0000gn/T/ipykernel_31246/2343508439.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historic["sell_signal"] = historic.rsi > rsi_upper
[I 2025-09-12 07:41:23,212] Trial 0 fin

In [19]:
study.best_params

{'rsi_window': 24,
 'rsi_lower': 19,
 'rsi_upper': 68,
 'stop_loss': 0.13239067858483725,
 'take_profit': 0.06493924980913501,
 'n_shares': 311}

In [20]:
study.best_value

0.08621993762643054

In [ ]:
cash, len(active_long_positions)

(1175409.6117202505, 0)